# Recuperar informações da Wikipedia

In [ ]:
import requests

In [ ]:
from requests.exceptions import ConnectionError

In [ ]:
def download_html(url, numero_tentativas=2):
    print("Realizando o download da página:", url)
    try:
        req = requests.get(url)
        if req.status_code != 200:
            if numero_tentativas > 0:
                print("Não foi possível realizar o download. Erro:", req.status_code)
                print("\nRealizando nova tentativa:")
                return download_html(url, numero_tentativas - 1)
            else:
                print("Número de tentativas excedidas. Erro: {}".format(req.status_code))
                html = None
                return html
        html = req.text
        return html
    except ConnectionError as e:
        print("Erro no download:", e)
        html = None

In [ ]:
html = download_html("https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil")

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
bs = BeautifulSoup(html, "html.parser")

In [ ]:
print(bs.prettify())

In [ ]:
print(bs.h1.text)

In [ ]:
bs.h1

## Recuperar todos os links disponíveis nessa página

In [ ]:
bs.a

Note que apenas um único link foi retornado. Para retornar todos precisamos utilizar um método chamado ```findall()```.

In [ ]:
links = bs.findAll('a')
print(links)

In [ ]:
print(type(links))

Note que além dos links, tem outras informações.

In [ ]:
links[3]

Para imprimir/salvar apenas os links, precisamos iterar na lista que foi criada para pegar apenas o conteúdo do href.

In [ ]:
for link in links:
    print(link.get("href"))

É importante verificar que estamos trazendo todos os links (internos e externos) com essa abordagem.

## Recuperar uma tabela

Como vimos, iremos recuperar a tabela que está disponível na página, que lista todos os estados do Brasil, adicionando outras informações, como a abreviação, capital, area, população, mortalidade infantil, entre outras.

Existem duas formas de recuperarmos a tabela correta. A primeira, seria recuperar todas as tabelas disponível, iterar em cada uma delas e verificar a classe que ela pertecem.

A outra forma, é mais rápida e simples. No navegador Chrome, abrar a página em questão, clique com o Botão Direito em cima da tabela que deseja recuperar e depois clique em Inspecionar (Ctrl + Shift + I) e copie o nome da classe.

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center; font-size:85%">
```

Iremos utilizar ```wikitable sortable jquery-tablesorter``` para recuperar somente essa tabela.

In [ ]:
tabela = bs.findAll('table', 
                 class_='wikitable sortable')

In [ ]:
print(len(tabela))
tabela

Ótimo! Já temos a tabela recuperada.

O que iremos fazer agora é extrair as informações e adicionar em um DataFrame. Para isso precisamos iterar em cada linha (```tr```) e então atribuir cada elemento de tr (```td```) à uma variável e então adicionar a uma lista.

In [ ]:
colunas = [th.text.lower()[:-1] for th in tabela[0].findAll('tr')[0].findAll('th')]

In [ ]:
colunas

Antes de renomear o nome de cada coluna, iremos remover as acentuações para facilitar a criação das variáveis.

Essa função retorna a forma normal da string (sem acentuação). As opções são:

| Forma | Descrição |
| ---- | ----- |
| **NFD** - Normalization Form Canonical Decomposition | Os caracteres são decompostos pela equivalência canônica e vários caracteres combinados são organizados em uma ordem específica.  | 
| **NFC** - Normalization Form Canonical Composition | Os caracteres são decompostos e, em seguida, recompostos pela equivalência canônica. |
| **NFKD** - Normalization Form Compatibility Decomposition | Os caracteres são decompostos por compatibilidade e vários caracteres combinados são organizados em uma ordem específica. |
| **NFKC** - Normalization Form Compatibility Composition | Os caracteres são decompostos por compatibilidade e depois recompostos pela equivalência canônica. |



https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize

In [ ]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [ ]:
colunas = [remover_acentos(nome) for nome in colunas]
colunas

In [ ]:
colunas[colunas.index('(% total) (2015)')] = "percentual_pib_2015"
colunas[colunas.index('pib per capita (r$) (2015)')] = 'pib_per_capita_reais_2015'

Remover os caracteres especiais ```(``` e ```)``` e substituir o espaco pelo caractere ```_```.

In [ ]:
colunas = [nome.replace('(', '') for nome in colunas]
colunas = [nome.replace(')', '') for nome in colunas]
colunas = [nome.replace(' ', '_') for nome in colunas]

In [ ]:
colunas

In [ ]:
linha = tabela[0].findAll('tr')[1].findAll('td')

In [ ]:
linha[0].find('a').get('href')

In [ ]:
linha[13].find(text=True)

In [ ]:
# Estruturando 
dados = {}
for nome in colunas:
    dados[nome] = []

In [ ]:
dados

In [ ]:
linhas = tabela[0].findAll('tr')
len(linhas)

In [ ]:
# Começa em 1, pois o 0 é o cabeçalho da tabela
for lin in range(1, len(linhas)):
    linha = tabela[0].findAll('tr')[lin].findAll('td')
    for i, chave in enumerate(colunas):
        if i == 0 and chave == 'bandeira':
            dados[chave].append(linha[i].find('a').get('href')[:-1])
        else:
            dados[chave].append(linha[i].find(text=True)[:-1])

In [ ]:
dados

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
df.head()

Agora, basta criar o DataFrame

In [ ]:
import pandas as pd

In [ ]:
df_final = df.drop('bandeira', axis=1)

In [ ]:
df_final.head()

### Realizando ajustes e conversões de tipos

Para realizarmos algumas análises será necessário realizar as conversões de tipo. 

Realizando uma copia do ```df_final``` para tratamento dos dados

In [ ]:
df_aux = df_final.copy()

Também será necessário selecionar as colunas que se deseja realizar o tratamento.

In [ ]:
print(len(df_aux.columns))

In [ ]:
colunas_removidas = ['abreviacao', 'sede_de_governo', 'unidade_federativa']

In [ ]:
aux_colunas = list(filter(lambda col: col not in colunas_removidas, df_aux.columns))

In [ ]:
print(len(aux_colunas))

Funções auxiliares para remover espaços e converter a pontuação.

In [ ]:
def remover_espacos(string):
    string = string.replace(u'%', '')
    string = string.replace(u'‰', '')
    string = string.replace(u'anos', '')
    return string.replace(u'\xa0', '')

In [ ]:
def converter_pontuacao(string):
    return string.replace(u',', '.')

Aplicando as funções criadas em cada uma das colunas definidas na variável ```colunas_float``` e ```colunas_int```.

In [ ]:
for coluna in aux_colunas:
    df_aux[coluna] = df_aux[coluna].apply(remover_espacos)
    df_aux[coluna] = df_aux[coluna].apply(converter_pontuacao)

In [ ]:
df_aux.head()

In [ ]:
print(type(df_aux.area_km2[0]))

Temos que realizar as conversões de tipo, para isso iremos utilizar o método do Pandas que realiza a conversão de maneira automatica seja para um tipo numérico.

In [ ]:
for coluna in aux_colunas:
    df_aux[coluna] = df_aux[coluna].apply(pd.to_numeric)

In [ ]:
df_aux.dtypes

Visualizando o DataFrame Final

In [ ]:
df_aux.head()

In [ ]:
df_aux.describe()

In [ ]:
# Essa opção do pandas format os tipos floats com duas casas decimais.
pd.options.display.float_format = "{:.2f}".format

In [ ]:
df_aux.describe()